In [3]:
import pandas as panda

In [4]:
from matplotlib import pyplot as plt

In [5]:
dataframe = panda.read_csv("Ruter_data.csv", sep=';')

In [6]:
dataframe.head()

,TurId,Dato,Fylke,Område,Kommune,Holdeplass_Fra,Holdeplass_Til,Linjetype,Linjefylke,Linjenavn,Linjeretning,Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra,Tidspunkt_Faktisk_Avgang_Holdeplass_Fra,Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra,Tidspunkt_Planlagt_Avgang_Holdeplass_Fra,Kjøretøy_Kapasitet,Passasjerer_Ombord
0,15006-2020-08-10T10:24:00+02:00,10/08/2020,Viken,Vest,Bærum,Nordliveien,Tjernsmyr,Lokal,Viken,150,0,10:53:53,10:53:59,10:53:00,10:53:00,112,5
1,15002-2020-08-15T12:54:00+02:00,15/08/2020,Viken,Vest,Bærum,Nadderud stadion,Bekkestua bussterminal (Plattform C),Lokal,Viken,150,0,13:12:20,13:12:26,13:12:00,13:12:00,112,5
2,15004-2020-08-03T09:54:00+02:00,03/08/2020,Viken,Vest,Bærum,Ringstabekkveien,Skallum,Lokal,Viken,150,0,10:18:56,10:19:21,10:19:00,10:19:00,112,6
3,15003-2020-07-27T13:00:00+02:00,27/07/2020,Viken,Vest,Bærum,Gruvemyra,Gullhaug,Lokal,Viken,150,1,13:52:04,13:52:26,13:51:00,13:51:00,112,10
4,15002-2020-08-27T07:15:00+02:00,27/08/2020,Viken,Vest,Bærum,Lysaker stasjon (Plattform A),Tjernsmyr,Lokal,Viken,150,1,07:34:13,07:34:53,07:33:00,07:33:00,112,10


In [7]:
dataframe.shape

(6000, 17)

In [8]:
thresold_max = dataframe['Passasjerer_Ombord'].quantile(0.90)

In [9]:
thresold_max

13.0

In [15]:
thresold_min = dataframe['Passasjerer_Ombord'].quantile(0.10)

In [16]:
dataframe[dataframe['Passasjerer_Ombord']< thresold_min]

,TurId,Dato,Fylke,Område,Kommune,Holdeplass_Fra,Holdeplass_Til,Linjetype,Linjefylke,Linjenavn,Linjeretning,Tidspunkt_Faktisk_Ankomst_Holdeplass_Fra,Tidspunkt_Faktisk_Avgang_Holdeplass_Fra,Tidspunkt_Planlagt_Ankomst_Holdeplass_Fra,Tidspunkt_Planlagt_Avgang_Holdeplass_Fra,Kjøretøy_Kapasitet,Passasjerer_Ombord
11,21005-2020-08-17T17:55:00+02:00,17/08/2020,Viken,Vest,Bærum,Haslum skole,Griniveien,Lokal,Viken,150,1,18:35:31,18:35:56,18:29:00,18:29:00,112,-2
15,15003-2020-08-08T19:18:00+02:00,08/08/2020,Viken,Vest,Bærum,Guldbrandslia,Sleiverudåsen,Lokal,Viken,150,1,20:04:55,20:05:01,20:02:00,20:02:00,112,-2
17,15002-2020-07-28T22:18:00+02:00,28/07/2020,Viken,Vest,Bærum,Steinskogen gravlund,Stein gård,Lokal,Viken,150,1,22:54:18,22:54:22,22:53:00,22:53:00,112,-5
18,15002-2020-08-03T21:24:00+02:00,03/08/2020,Oslo,Indre By,Sentrum,Nationaltheatret (plf F i Munkedamsv m Stortin...,St. Olavs plass (mot Storgata),Lokal,Viken,150,0,22:11:36,22:11:41,22:04:00,22:04:00,112,-7
30,26120-2020-08-14T08:41:00+02:00,14/08/2020,Viken,Vest,Asker,Torsrudveien,Bølstad terrasse,Lokal,Viken,261,0,09:07:06,09:07:09,09:07:00,09:07:00,112,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5961,43007-2020-06-19T14:25:00+02:00,19/06/2020,Viken,Nordøst,Eidsvoll,Eidsvoll verk stasjon (Plattform 1),Håpet,Lokal,Viken,430,1,15:03:12,15:03:56,14:56:00,14:56:00,106,-15
5965,43003-2020-06-23T20:55:00+02:00,23/06/2020,Viken,Nordøst,Eidsvoll,Hammerstad,Ås skole,Lokal,Viken,430,1,21:38:35,21:38:39,21:38:00,21:38:00,106,-16
5989,10010-2020-06-28T13:00:00+02:00,28/06/2020,Viken,Nordøst,Lillestrøm,Vestbygata,Måsasvingen vest,Lokal,Viken,100,1,13:56:05,13:56:13,13:56:00,13:56:00,151,-2
5991,12004-2020-08-16T18:26:00+02:00,16/08/2020,Viken,Nordøst,Lørenskog,Ahus,Vittenberg,Lokal,Viken,310,1,18:39:51,18:41:27,18:39:00,18:39:00,72,-3


In [20]:
new_dataframe1=dataframe[(dataframe['Passasjerer_Ombord']<thresold_max) & (dataframe['Passasjerer_Ombord']>thresold_min)]

In [21]:
new_dataframe1.shape 

(4695, 17)